In [1]:
import random
import numpy as np
from tinysmpc.tinysmpc import VirtualMachine, PrivateScalar
from tinysmpc.fixed_point import fixed_point, float_point

In [2]:
N = 4
Q = 2657003489534545107915232808830590043

In [3]:
def additive_share(secret, N):
    shares = [random.randrange(Q) for _ in range(N - 1)]
    shares += [(secret - sum(shares)) % Q]
    return shares


def additive_reconstruct(shares):
    return sum(shares) % Q


def share_tensor(tensor, n):
    if tensor.ndim == 1:
        tensor = np.expand_dims(tensor, axis=1)

    random_values = [random.randrange(Q) for _ in range(np.prod((tensor.shape + (n - 1,))))]
    random_shares = np.array(random_values).reshape(n - 1, tensor.shape[0], tensor.shape[1])
    n_shares = np.concatenate([random_shares, [(tensor - random_shares.sum(axis=0)) % Q]])
    return n_shares


def additive_reconstruct_tensor(shares):
    secrets = shares.sum(axis=0) % Q
    return secrets

# Fixed Point Conversion Testing
### Single Values

In [4]:
test_float_point = 1.2345

test_fixed_point = fixed_point(test_float_point)

test_fixed_point_shares = additive_share(test_fixed_point, 3)

test_fixed_point_recon = additive_reconstruct(test_fixed_point_shares)

test_reconstructed_float_point = float_point(test_fixed_point)

print(f"Test Float Point: {test_float_point}")
print(f"Test Fixed Point (Converted): {test_fixed_point}")
print(f"Test Fixed Point Shares: {test_fixed_point_shares}")
print(f"Test Fixed Point Additive Reconstructed: {test_fixed_point_recon}")
print(f"Test Float Point Reconstructed: {test_reconstructed_float_point}")

Test Float Point: 1.2345
Test Fixed Point (Converted): 123450000
Test Fixed Point Shares: [2292647927156271777280547472277934142, 153414946179535993064733405463354015, 210940616198737337569951931212751886]
Test Fixed Point Additive Reconstructed: 123450000
Test Float Point Reconstructed: 1.2345


### Tensor Values

In [5]:
fixedPoint = np.vectorize(fixed_point)
floatPoint = np.vectorize(float_point)

float_point_secret = np.random.rand(4, 3)

fixed_point_secret = fixedPoint(float_point_secret)
float_point_secret_reconstructed = floatPoint(fixed_point_secret)

print(f"Float Point Secret\n{float_point_secret}\n")
print(f"Fixed Point Secret (Converted)\n{fixed_point_secret}\n")
print(f"Float Point Secret Reconstructed\n{float_point_secret_reconstructed}\n")

Float Point Secret
[[0.79968989 0.66124635 0.70672638]
 [0.48489617 0.67780184 0.87376581]
 [0.70964431 0.84952069 0.59766224]
 [0.88509929 0.56674216 0.77435297]]

Fixed Point Secret (Converted)
[[79968988 66124635 70672637]
 [48489617 67780183 87376581]
 [70964431 84952068 59766224]
 [88509929 56674216 77435296]]

Float Point Secret Reconstructed
[[0.79968988 0.66124635 0.70672637]
 [0.48489617 0.67780183 0.87376581]
 [0.70964431 0.84952068 0.59766224]
 [0.88509929 0.56674216 0.77435296]]



### Integer Tensor Additive Sharing Test

In [6]:
secret_int = np.random.randint(1, 20, (4, 3))
print(f"Secret\n{secret_int}\n")

test_shares = share_tensor(secret_int, N)
# print(f"Secret Shares: \n{test_shares}\n")

secret_reconstructed = additive_reconstruct_tensor(test_shares)

print(f"Reconstructed Secret\n{secret_reconstructed}\n")

Secret
[[ 2 15 18]
 [ 6 13 10]
 [18  7  1]
 [18 10  9]]

Reconstructed Secret
[[2 15 18]
 [6 13 10]
 [18 7 1]
 [18 10 9]]



### Float Tensor Additive Sharing Test

In [15]:
fixed_point_secret = fixedPoint(float_point_secret)
print(f"Fixed Point Secret\n{fixed_point_secret}\n")

fixed_point_test_shares = share_tensor(fixed_point_secret, N)
fixed_point_secret_reconstructed = additive_reconstruct_tensor(fixed_point_test_shares)

print(f"Secret Shares \n{fixed_point_test_shares}\n")
print(f"Reconstructed Fixed Point Secret\n{fixed_point_secret_reconstructed}\n")
print(f"Reconstructed Float Point Secret\n{floatPoint(fixed_point_secret_reconstructed)}\n")

Fixed Point Secret
[[79968988 66124635 70672637]
 [48489617 67780183 87376581]
 [70964431 84952068 59766224]
 [88509929 56674216 77435296]]

Secret Shares 
[[[2449176870901746276146223537418284200
   921615431078941823420907911101475773
   2515118223691653604197854397761331271]
  [724739347664024662781177928168272137
   1369356444147215129194621968035128546
   1608540646789960351166204491643362266]
  [2472361101297132520615464426118624484
   2211684131487887340492347087457223190
   1504256000183032074889062611651726715]
  [708735606702736837073622418719926716
   595803118338681439928594869582167234
   1321158731528577693592491329678156773]]

 [[225785247578094269806639134259646465
   1233375255562014506788155553276795718
   876194123092123793110868694997084689]
  [1389103600362321088158348260899882125
   155569768373940428525091976547558134
   498100001155619152450357895124992105]
  [357681807456591319583697184465324367
   1818364351574667119916192852181148259
   2161942984145019333279

In [8]:

parameters = [
    ["kofi", "ama", "kojo"],
    [40000, 15000, -20000],
]

"""
Secret sharing process
"""
#
aggregator = VirtualMachine("aggregator")

# Edge Devices
nodes = [VirtualMachine(node_id) for node_id in parameters[0]]

# Tensors of edge devices
node_values = [PrivateScalar(tensor, node) for tensor, node in zip(parameters[1], nodes)]

# Nodes with their local shares
exchanged_shares = []

for value in node_values:
    virtual_node = VirtualMachine("virtual_node")
    exchanged_shares.append(value.share(nodes).reconstruct(virtual_node).value)

# temp = VirtualMachine("temp")
for node in exchanged_shares:
    print(node)
#     print(node.reconstruct(temp))
#     print("\n\n")
#
# shared_sum = sum(exchanged_shares)
# shared_sum.reconstruct(aggregator)

40000
15000
-20000


In [14]:
tensor_1 = fixedPoint(np.random.rand(2, 3))
tensor_2 = fixedPoint(np.random.rand(2, 3))
tensor_3 = fixedPoint(np.random.rand(2, 3))

fl_node_weights = [
    ["kofi", "ama"],
    [tensor_1, tensor_2],
]

"""
Secret sharing process
"""
#
fl_server = VirtualMachine("fl_server")

# Edge Devices
fl_nodes = [VirtualMachine(node_id) for node_id in fl_node_weights[0]]

# Tensors of edge devices
fl_node_values = [PrivateScalar(tensor, node) for tensor, node in zip(fl_node_weights[1], fl_nodes)]

# Nodes with their local shares
fl_exchanged_shares = []

for value in fl_node_values:
    fl_exchanged_shares.append(value.share_tensor(fl_nodes, Q))

shared_sum = sum(fl_exchanged_shares)
shared_sum.reconstruct(fl_server)

print(f"Original Sum\n{shared_sum}\n")

original_sum = np.array(fl_node_weights[1]).sum(axis=0)

final_sum = np.array(fl_server.objects[-1].value)

print(f"Original Sum\n{original_sum}\n")
print(f"Shared Sum\n{final_sum}\n")
# print(f"Exchanged Shares\n{fl_exchanged_shares[0]}\n")

Original Sum
SharedScalar
 - Share([[2543567554313415048125257998269096493
  1691888855646298391950275914622336810
  1830464034164250698229332936586364301]
 [1802280360049732169445758504822475117
  923320525339267326461849518629645518
  651976620369297962072174125719122303]], 'ama', Q=2657003489534545107915232808830590043)
 - Share([[113435935221130059789974810615191351
  965114633888246715964956894224846181
  826539455370294409685899872346007141]
 [854723129484812938469474304101319635
  1733682964195277781453383290335299853
  2005026869165247145843058683294393360]], 'kofi', Q=2657003489534545107915232808830590043)

Original Sum
[[ 53697801  16592948 101781399]
 [ 93204709 134355328 182925620]]

Shared Sum
[[53697801 16592948 101781399]
 [93204709 134355328 182925620]]



In [10]:
print(tensor_1)

[89483306 49912214]


In [11]:
print(fl_node_values)

[PrivateScalar([89483306 49912214], 'kofi'), PrivateScalar([31957977 47438176], 'ama')]


In [12]:
print(fl_node_weights[1])

[array([89483306, 49912214]), array([31957977, 47438176])]
